# 目的
1-3で作成した手法を用いてより詳細なデータセットを作成する

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/home/yosuke-kurosu/pj09_10％/saunadataset.csv")

In [4]:
def catchinfo(number):
    # サウナ番号を指定してURLを取得する
    r = requests.get('https://sauna-ikitai.com/saunas/{}'.format(number))
    # html情報の取得
    soup = BeautifulSoup(r.content, "html.parser")
    #=====以下サウナ情報を取得する処理=====
    name = soup.find("span", {"class":"c-headline_string"}).string #サウナ名の取得
    iki = soup.find("div", {"class":"p-action_number js-ikitaiCounter"}).string #イキタイ度の取得
    act = soup.find("span", {"class":"p-localNav_count"}).string #サ活の取得
    temp1 = soup.find_all("div", {"class":"p-saunaSpecItem p-saunaSpecItem--sauna"})[0].text.replace(' ','').replace('\n','').split('度')[1] #サウナ温度の取得
    temp2 = man = soup.find_all("div", {"class":"p-saunaSpecItem p-saunaSpecItem--mizuburo"})[0].text.replace(' ','').replace('\n','').split('度')[1]  #水風呂温度の取得
    return [name, iki, act, temp1, temp2]

# 女性限定サウナ、男性限定サウナを考慮した関数に書き換える

In [5]:
sauna = catchinfo(2507)

In [6]:
sauna

['SaunaLab-サウナラボ-', '379', '125', '60', '-']

In [88]:
r = requests.get("https://sauna-ikitai.com/saunas/1706")
soup = BeautifulSoup(r.content, "html.parser")

In [89]:
#<div class="p-saunaSpecTable_name 
saunatable = soup.find_all("td", {"class":"p-saunaSpecTable_mark"})

In [90]:
table = []
for i, text in enumerate(saunatable):
    if i == 6:
        break;
    
    if "ok" in str(text):
        table.append('ok')
    elif "ng" in str(text):
        table.append('ng')
    else:
        table.append('-')

In [91]:
table

['ok', 'ng', 'ng', 'ok', 'ok', 'ok']

In [20]:
#<p class="p-saunaDetailHeader_target">女性専用</p>
target = soup.find("p", {"class":"p-saunaDetailHeader_target"}).string

In [22]:
# 女性専用　男性専用　対象：男女

In [188]:
# 今回のnotebookで更新した関数、男女のサウナ水風呂の温度を別々に取得、ロウリュ等の追加情報も取得するようになった

def catchinfo(number):
    """
    入力:
    -サウナイキタイのページindex
    出力:
    -サウナ情報
    サウナ名、イキタイ度、サ活数、男サウナ温度、男水風呂温度、女サウナ温度、女水風呂温度etc
    """
    
    # サウナ番号を指定してURLを取得する
    r = requests.get('https://sauna-ikitai.com/saunas/{}'.format(number))
    # html情報の取得
    soup = BeautifulSoup(r.content, "html.parser")
    #=====基本情報を取得する処理=====
    name = soup.find("span", {"class":"c-headline_string"}).string #サウナ名の取得
    iki = soup.find("div", {"class":"p-action_number js-ikitaiCounter"}).string #イキタイ度の取得
    act = soup.find("span", {"class":"p-localNav_count"}).string #サ活の取得
    target = soup.find("p", {"class":"p-saunaDetailHeader_target"}).string
    mantemp1 = soup.find_all("div", {"class":"p-saunaSpecItem p-saunaSpecItem--sauna"})[0].text.replace(' ','').replace('\n','').split('度')[1] #男性用サウナ温度の取得
    mantemp2 = soup.find_all("div", {"class":"p-saunaSpecItem p-saunaSpecItem--mizuburo"})[0].text.replace(' ','').replace('\n','').split('度')[1] #男性用水風呂温度の取得
    womantemp1 = soup.find_all("div", {"class":"p-saunaSpecItem p-saunaSpecItem--sauna"})[1].text.replace(' ','').replace('\n','').split('度')[1] #女性用サウナ温度の取得
    womantemp2 = soup.find_all("div", {"class":"p-saunaSpecItem p-saunaSpecItem--mizuburo"})[1].text.replace(' ','').replace('\n','').split('度')[1] #女性用水風呂温度の取得
    #=====ロウリュ等の追加情報を取得する処理=====
    saunatable = soup.find_all("td", {"class":"p-saunaSpecTable_mark"})
    apendinfo = []
    for i, text in enumerate(saunatable):
        if "ok" in str(text):
            apendinfo.append('ok')
        elif "ng" in str(text):
            apendinfo.append('ng')
        else:
            apendinfo.append('-')    
    return [name, iki, act, target, mantemp1, mantemp2, womantemp1, womantemp2, apendinfo[0],\
           apendinfo[1], apendinfo[2], apendinfo[3], apendinfo[4], apendinfo[5], apendinfo[6],\
           apendinfo[7], apendinfo[8], apendinfo[9], apendinfo[10], apendinfo[11]]

In [192]:
saunalist = []
for i in tqdm(range(1,5880)):
    try:
        saunalist.append(catchinfo(i))
    except:
        pass



  0%|          | 0/5879 [00:00<?, ?it/s]

  0%|          | 1/5879 [00:01<1:49:29,  1.12s/it]

  0%|          | 2/5879 [00:01<1:33:58,  1.04it/s]

  0%|          | 3/5879 [00:02<1:23:41,  1.17it/s]

  0%|          | 4/5879 [00:02<1:13:12,  1.34it/s]

  0%|          | 5/5879 [00:03<1:12:22,  1.35it/s]

  0%|          | 6/5879 [00:03<54:54,  1.78it/s]  

  0%|          | 7/5879 [00:04<1:00:50,  1.61it/s]

  0%|          | 8/5879 [00:05<1:02:42,  1.56it/s]

  0%|          | 9/5879 [00:05<1:05:44,  1.49it/s]

  0%|          | 10/5879 [00:06<1:00:35,  1.61it/s]

  0%|          | 11/5879 [00:06<59:12,  1.65it/s]  

  0%|          | 12/5879 [00:07<59:19,  1.65it/s]

  0%|          | 13/5879 [00:08<58:44,  1.66it/s]

  0%|          | 14/5879 [00:08<1:03:20,  1.54it/s]

  0%|          | 15/5879 [00:09<1:00:52,  1.61it/s]

  0%|          | 16/5879 [00:10<1:01:40,  1.58it/s]

  0%|          | 17/5879 [00:10<1:01:16,  1.59it/s]

  0%|          | 18/5879 [00:11<1:03:12,  1.55it/s]

  0%|          

In [193]:
columns = ['サウナ名', 'イキタイ', 'サ活', 'target', '男サウナ', '男水風呂', '女サウナ', '女水風呂',\
          '男ロウリュ', '男オート', '男セルフ', '男外気浴', '男休憩', '男イオン',\
          '女ロウリュ', '女オート', '女セルフ', '女外気浴', '女休憩', '女イオン']
df = pd.DataFrame(saunalist)
df.columns = columns
df.to_csv('saunalist_0930.csv')

In [194]:
df = pd.read_csv("/home/yosuke-kurosu/pj09_10％/saunalist_0930.csv")

In [199]:
df = df.drop('Unnamed: 0', axis=1)

In [200]:
df.to_csv("sauna_0930.csv", index=False)

In [203]:
df.head(100)

,サウナ名,イキタイ,サ活,target,男サウナ,男水風呂,女サウナ,女水風呂,男ロウリュ,男オート,男セルフ,男外気浴,男休憩,男イオン,女ロウリュ,女オート,女セルフ,女外気浴,女休憩,女イオン
0,稚内温泉 童夢,0,4,対象：男女,90,15,?,?,ng,ng,ng,ok,ok,ok,-,-,-,-,-,-
1,天然温泉天北の湯ドーミーイン稚内,2,4,対象：男女,96,16,?,?,ng,ng,ng,ok,ng,-,-,-,-,-,-,-
2,稚内天然温泉 港のゆ,5,5,対象：男女,92,20,90,17,ng,ng,ng,ok,ok,ok,ng,ng,ng,ok,ok,ok
3,稚内グランドホテル,0,0,対象：-,?,?,?,?,-,-,-,-,-,-,-,-,-,-,-,-
4,ホテル奥田屋,0,0,対象：-,?,?,?,?,-,-,-,-,-,-,-,-,-,-,-,-
5,礼文島温泉 うすゆきの湯,3,5,対象：男女,90,18,90,12,ng,ng,ng,ng,ok,ng,ng,ng,ng,ng,ok,ng
6,利尻富士温泉,12,7,対象：男女,100,7,?,?,ng,ng,ng,ok,ok,ok,-,-,-,-,-,-
7,はまとんべつ温泉ウイング,0,0,対象：男女,90,18,90,18,ng,ng,ng,ng,ng,-,ng,ng,ng,ng,ng,-
8,ホテル豊富,0,1,対象：男女,-,-,91,14,ng,ng,ng,ng,ng,ng,ng,ng,ng,ng,ng,ng
9,てしお温泉夕映,1,5,対象：男女,90,20,90,16,ng,ng,ng,ng,ok,ng,ng,ng,ng,ng,ok,ng
